In [1]:
!pip install -e .. > /dev/null 2>&1

In [2]:
import os
import json
import gdown
import shutil

from astrohack.gdown_utils import gdown_data
from astrohack.gdown_utils import build_folder_structure

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20, 20]

In [3]:
# Build folder structure & download 
import astrohack

base_name = 'ea25_cal_small_'

datafolder = 'data'
resultsfolder = 'results'

build_folder_structure(datafolder, resultsfolder)

'''
astrohack.gdown_utils.download('ea25_cal_small_before_fixed.split.ms', folder=datafolder, unpack=True)
astrohack.gdown_utils.download('ea25_cal_small_after_fixed.split.ms', folder=datafolder, unpack=True)
'''
astrohack.gdown_utils.download(file='extract_holog_verification.json')
astrohack.gdown_utils.download(file='holog_numerical_verification.json')


In [4]:
from astrohack.astrohack_client import astrohack_local_client

log_parms = {'log_level':'DEBUG'}

client = astrohack_local_client(cores=2, memory_limit='8GB', log_parms=log_parms)

2023-07-20 15:05:14,992 - INFO     - astrohack - (_check_parms.py:135) - [_check_logger_parms]: Setting default log_to_term to True.
2023-07-20 15:05:14,994 - INFO     - astrohack - (_check_parms.py:135) - [_check_logger_parms]: Setting default log_to_file to False.
2023-07-20 15:05:14,996 - INFO     - astrohack - (_check_parms.py:135) - [_check_logger_parms]: Setting default log_file to hack_.
2023-07-20 15:05:14,998 - INFO     - astrohack - (_check_parms.py:135) - [_check_worker_logger_parms]: Setting default log_to_term to False.
2023-07-20 15:05:14,999 - INFO     - astrohack - (_check_parms.py:135) - [_check_worker_logger_parms]: Setting default log_to_file to False.
2023-07-20 15:05:15,000 - INFO     - astrohack - (_check_parms.py:135) - [_check_worker_logger_parms]: Setting default log_file to hack_.
2023-07-20 15:05:15,001 - INFO     - astrohack - (_check_parms.py:135) - [_check_worker_logger_parms]: Setting default log_level to INFO.
2023-07-20 15:05:16,591 - INFO     - astroha

In [5]:
def test_holog_obs_dictionary(holog_obs_dict):
    import os
    import json
    import copy
    
    import numpy as np

    from astrohack._utils._tools import _jsonify

    ref_holog_obj = {}
    ref_holog_obj = copy.deepcopy(holog_obs_dict)

    _jsonify(ref_holog_obj)

    with open(".holog_obs_dict.json") as json_file:
        holog_obj = json.load(json_file)
                          
    assert holog_obj == ref_holog_obj, "Error: holog_obs_descrition dictionary has changes unexpectedly."
    
def test_holog_diagnostics(json_data, tolerance=1e-7):
    import json
    
    with open("holog_numerical_verification.json") as file:
        reference_dict = json.load(file)
        
    cell_size = reference_dict["vla"]['cell_size'][1]
    grid_size = float(reference_dict["vla"]['grid_size'][1])
    
    json_data['cell_size'] = np.abs(float(json_data['cell_size']))
    
    cell_size = np.abs(float(cell_size))
    
    assert relative_difference(json_data['cell_size'], cell_size) < tolerance, "Unexpected change in cell_size occured."
    assert relative_difference(np.sqrt(int(json_data['n_pix'])), grid_size) < tolerance, "Unexpected change in grid_size occured."

    
def test_center_pixel(file, antenna, ddi, reference_center_pixels, number_of_digits=7):
    from astrohack.dio import open_image
    
    mds = open_image(file)[antenna][ddi]
    
    aperture_shape = mds.APERTURE.values.shape[-2], mds.APERTURE.values.shape[-1]
    beam_shape = mds.BEAM.values.shape[-2], mds.BEAM.values.shape[-1]    
    
    aperture_center_pixels = np.squeeze(mds.APERTURE.values[..., aperture_shape[0]//2, aperture_shape[1]//2])
    beam_center_pixels = np.squeeze(mds.BEAM.values[..., beam_shape[0]//2, beam_shape[1]//2])
    
    aperture_ref = list(map(complex, reference_center_pixels['aperture']))
    beam_ref = list(map(complex, reference_center_pixels['beam']))
    
    for i in range(len(aperture_ref)):        
        assert relative_difference(
            aperture_ref[i].real, 
            aperture_center_pixels[i].real
        ) < 1e-6, "There has been a shift in aperture center pixel value(s)"
        
        assert relative_difference(
            beam_ref[i].real, 
            beam_center_pixels[i].real
        ) < 1e-6, "There has been a shift in beam center pixel value(s)"
                
        assert relative_difference(
            aperture_ref[i].imag, 
            aperture_center_pixels[i].imag
        ) < 1e-6, "There has been a shift in aperture center pixel value(s)"
        assert relative_difference(
            beam_ref[i].imag, 
            beam_center_pixels[i].imag
        ) < 1e-6, "There has been a shift in beam center pixel value(s)"
        
              
def get_center_pixel(file, antenna, ddi):
    from astrohack.dio import open_image
    
    mds = open_image(file)[antenna][ddi]
    
    aperture_shape = mds.APERTURE.values.shape[-2], mds.APERTURE.values.shape[-1]
    beam_shape = mds.BEAM.values.shape[-2], mds.BEAM.values.shape[-1]    
    
    aperture_center_pixels = mds.APERTURE.values[..., aperture_shape[0]//2, aperture_shape[1]//2]
    beam_center_pixels = mds.BEAM.values[..., beam_shape[0]//2, beam_shape[1]//2]
    
    return np.squeeze(aperture_center_pixels), np.squeeze(beam_center_pixels)

def compare_float_values(result, reference, decimals):
    result_list = list(map(str, str(result)))
    reference_list = list(map(str, str(reference)))
    
    return result_list[:decimals] == reference_list[:decimals]

def relative_difference(result, expected):
    return 2*np.abs(result - expected)/(abs(result) + abs(expected))

In [8]:
from astrohack.extract_pointing import extract_pointing

before_ms = datafolder+'/'+ base_name + 'before_fixed.split.ms'
after_ms =  datafolder+'/'+ base_name + 'after_fixed.split.ms'

before_point = resultsfolder+'/before.split.point.zarr'
after_point = resultsfolder+'/before.split.point.zarr'

pnt_mds = extract_pointing(
      ms_name=before_ms,
      point_name=before_point,
      parallel=False,
      overwrite=True
)

extract_pointing(
      ms_name=after_ms,
      point_name=after_point,
      parallel=False,
      overwrite=True
)

2023-07-20 15:06:25,674 - WARNING  - astrohack - (_dio.py:39) - [extract_pointing]: results/before.split.point.zarr will be overwritten.
Successful readonly open of usernoread-locked table data/ea25_cal_small_before_fixed.split.ms/ANTENNA: 8 columns, 3 rows
2023-07-20 15:06:25,699 - DEBUG    - astrohack - (_extract_point.py:85) - Holography Scans Times {0: {8: array([5.16975363e+09, 5.16975395e+09]), 9: array([5.16975395e+09, 5.16975427e+09]), 10: array([5.16975428e+09, 5.16975460e+09]), 12: array([5.16975462e+09, 5.16975494e+09]), 13: array([5.16975494e+09, 5.16975526e+09]), 14: array([5.16975526e+09, 5.16975558e+09]), 16: array([5.16975560e+09, 5.16975593e+09]), 17: array([5.16975593e+09, 5.16975625e+09]), 18: array([5.16975625e+09, 5.16975657e+09]), 23: array([5.16975679e+09, 5.16975711e+09]), 24: array([5.16975711e+09, 5.16975743e+09]), 25: array([5.16975744e+09, 5.16975776e+09]), 27: array([5.16975778e+09, 5.16975810e+09]), 28: array([5.16975810e+09, 5.16975842e+09]), 29: array([5

{'point_meta_ds': <xarray.Dataset>
 Dimensions:  ()
 Data variables:
     *empty*
 Attributes:
     mapping_state_ids:  [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, ...,
 'ant_ea04': <xarray.Dataset>
 Dimensions:              (time: 114465, az_el: 2, lm: 2)
 Coordinates:
   * time                 (time) float64 5.17e+09 5.17e+09 ... 5.17e+09 5.17e+09
 Dimensions without coordinates: az_el, lm
 Data variables:
     DIRECTION            (time, az_el) float64 dask.array<chunksize=(57233, 1), meta=np.ndarray>
     DIRECTIONAL_COSINES  (time, lm) float64 dask.array<chunksize=(57233, 1), meta=np.ndarray>
     ENCODER              (time, az_el) float64 dask.array<chunksize=(57233, 1), meta=np.ndarray>
     POINTING_OFFSET      (time, az_el) float64 dask.array<chunksize=(57233, 1), meta=np.ndarray>
     TARGET               (time, az_el) float64 dask.array<chunksize=(57233, 1), meta=np.ndarray>
 Attributes:
     ant_name:                ea04
     mapping_scans_obs_dict:  [{'ddi_0': {},

In [34]:
pnt_mds['ant_ea25'].DIRECTIONAL_COSINES.values

ValueError: cannot reshape array of size 56663 into shape (56664,1)

In [7]:
from astrohack.extract_holog import extract_holog
import numpy as np

before_holog = resultsfolder+'/before.split.holog.zarr'
after_holog = resultsfolder+'/after.split.holog.zarr'

with open("extract_holog_verification.json") as file:
    json_dict = json.load(file)
    
holog_obs_dict = json_dict["vla"]

extract_holog(
    ms_name=before_ms, 
    point_name=before_point,
    holog_name=before_holog,
    ddi=[0],
    data_column='CORRECTED_DATA',
    parallel=False,
    overwrite=True
)

#test_holog_obs_dictionary(holog_obs_dict["before"])

with open(resultsfolder+'/before.split.holog.zarr/.holog_attr') as attr_file:
    holog_attr = json.load(attr_file)
    
#test_holog_diagnostics(
#    json_data=holog_attr,
#    tolerance=2e-5
#)


extract_holog(
    ms_name=after_ms,
    point_name=after_point,
    holog_name=after_holog,
    data_column='CORRECTED_DATA',
    parallel=False,
    overwrite=True
)

#test_holog_obs_dictionary(holog_obs_dict["after"])


with open(resultsfolder+'/after.split.holog.zarr/.holog_attr') as attr_file:
    holog_attr = json.load(attr_file)
    
#test_holog_diagnostics(
#    json_data=holog_attr,
#    tolerance=2e-5
#)


2023-07-20 15:05:20,784 - WARNING  - astrohack - (_dio.py:39) - [extract_holog]: results/before.split.holog.zarr will be overwritten.
2023-07-20 15:05:20,810 - INFO     - astrohack - (extract_holog.py:238) - [extract_holog]: holog_obs_dict: 
{ 'map_0': { 'ant': { 'ea06': array(['ea04'], dtype='<U4'),
                      'ea25': array(['ea04'], dtype='<U4')},
             'scans': array([ 8,  9, 10, 12, 13, 14, 16, 17, 18, 23, 24, 25, 27, 28, 29, 31, 32,
       33, 38, 39, 40, 42, 43, 44, 46, 47, 48, 53, 54, 55, 57])}}
2023-07-20 15:05:20,814 - INFO     - astrohack - (extract_holog.py:339) - [extract_holog]: Processing ddi: 0, scans: [ 8  9 10 12 13 14 16 17 18 23 24 25 27 28 29 31 32 33 38 39 40 42 43 44
 46 47 48 53 54 55 57]
[[nan nan]
 [nan nan]
 [nan nan]
 ...
 [nan nan]
 [nan nan]
 [nan nan]]
[[nan nan]
 [nan nan]
 [nan nan]
 ...
 [nan nan]
 [nan nan]
 [nan nan]]
Successful readonly open of default-locked table data/ea25_cal_small_before_fixed.split.ms/ANTENNA: 8 columns, 3 rows

/home/jhoskins/miniconda3/envs/holo/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jhoskins/miniconda3/envs/holo/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jhoskins/miniconda3/envs/holo/lib/python3.8/site-packages/erfa/core.py:4613: RuntimeWarning: invalid value encountered in ld
  p1 = ufunc.ld(bm, p, q, e, em, dlim)
/home/jhoskins/miniconda3/envs/holo/lib/python3.8/site-packages/erfa/core.py:19005: RuntimeWarning: invalid value encountered in anp
  c_retval = ufunc.anp(a)
/home/jhoskins/miniconda3/envs/holo/lib/python3.8/site-packages/erfa/core.py:4029: RuntimeWarning: invalid value encountered in atioq
  aob, zob, hob, dob, rob = ufunc.atioq(ri, di, astrom)


2023-07-20 15:05:25,013 - INFO     - astrohack - (_extract_holog.py:367) - Writing holog file to results/before.split.holog.zarr
2023-07-20 15:05:25,091 - INFO     - astrohack - (_extract_holog.py:367) - Writing holog file to results/before.split.holog.zarr
2023-07-20 15:05:25,162 - INFO     - astrohack - (_extract_holog.py:133) - Finished extracting holography chunk for ddi: 0 holog_map_key: map_0
2023-07-20 15:05:25,167 - INFO     - astrohack - (extract_holog.py:389) - [extract_holog]: Finished processing
2023-07-20 15:05:25,217 - WARNING  - astrohack - (_dio.py:39) - [extract_holog]: results/after.split.holog.zarr will be overwritten.
2023-07-20 15:05:25,248 - INFO     - astrohack - (extract_holog.py:238) - [extract_holog]: holog_obs_dict: 
{ 'map_0': { 'ant': { 'ea06': array(['ea04'], dtype='<U4'),
                      'ea25': array(['ea04'], dtype='<U4')},
             'scans': array([ 8,  9, 10, 12, 13, 14, 16, 17, 18, 23, 24, 25, 27, 28, 29, 31, 32,
       33, 38, 39, 40, 42, 4

/home/jhoskins/miniconda3/envs/holo/lib/python3.8/site-packages/erfa/core.py:4613: RuntimeWarning: invalid value encountered in ld
  p1 = ufunc.ld(bm, p, q, e, em, dlim)
/home/jhoskins/miniconda3/envs/holo/lib/python3.8/site-packages/erfa/core.py:19005: RuntimeWarning: invalid value encountered in anp
  c_retval = ufunc.anp(a)
/home/jhoskins/miniconda3/envs/holo/lib/python3.8/site-packages/erfa/core.py:4029: RuntimeWarning: invalid value encountered in atioq
  aob, zob, hob, dob, rob = ufunc.atioq(ri, di, astrom)


[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Successful readonly open of default-locked table data/ea25_cal_small_after_fixed.split.ms/ANTENNA: 8 columns, 3 rows
Successful readonly open of default-locked table data/ea25_cal_small_after_fixed.split.ms/OBSERVATION: 9 columns, 1 rows
2023-07-20 15:05:26,593 - INFO     - astrohack - (_extract_holog.py:367) - Writing holog file to results/after.split.holog.zarr
2023-07-20 15:05:26,677 - INFO     - astrohack - (_extract_holog.py:367) - Writing holog file to results/after.split.holog.zarr
2023-07-20 15:05:26,750 - INFO     - astrohack - (_extract_holog.py:133) - Finished extracting holography chunk for ddi: 0 holog_map_key: map_0
2023-07-20 15:05:26,755 - INFO     - astrohack - (extract_holog.py:339) - [extract_holog]: Processing ddi: 1, scans: [ 8  9 10 12 13 14 16 17 18 23 24 25 27 28 29 31 32 33 38 39 40 42 43 44
 46 47 48 53 54 55 57]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 .

In [ ]:
import numpy as np
from astrohack import holog

with open("holog_numerical_verification.json") as file:
    reference_dict = json.load(file)

grid_interpolation_mode = 'linear' #'nearest' 'linear' 'cubic'
chan_average = True
scan_average = True

holog(
    holog_name=beforeholog, 
    padding_factor=50, 
    grid_interpolation_mode=grid_interpolation_mode,
    chan_average = chan_average,
    scan_average = scan_average,
    overwrite=True,
    phase_fit=True,
    apply_mask=True,
    to_stokes=True,
    parallel=True
)

test_center_pixel(file="results/before.split.image.zarr", antenna="ant_ea25", ddi="ddi_0", reference_center_pixels=reference_dict["vla"]["pixels"]["before"])



holog(
    holog_name=afterholog, 
    padding_factor=50, 
    grid_interpolation_mode=grid_interpolation_mode,
    chan_average = chan_average,
    scan_average = scan_average,
    overwrite=True,
    phase_fit=True,
    apply_mask=True,
    to_stokes=True,
    parallel=True
)

test_center_pixel(file="results/after.split.image.zarr", antenna="ant_ea25", ddi="ddi_0", reference_center_pixels=reference_dict["vla"]["pixels"]["after"])



In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import scipy
import matplotlib.patches as patches

from scipy import constants

from astrohack.dio import open_image

beforeimage = resultsfolder+'/before.split.image.zarr'
afterimage = resultsfolder+'/after.split.image.zarr'

plt.close('all')

chan = 0

ds = open_image("results/before.split.image.zarr")['ant_ea25']['ddi_0']

wavelength = scipy.constants.speed_of_light/ds.chan.values[chan]
l = ds.l.values
m = ds.m.values
u = ds.u.values*wavelength
v = ds.v.values*wavelength

print(u.min(),u.max())

plt.figure()
plt.imshow(np.abs(ds.BEAM[0,chan,0,:,:]),extent=[l.min(), l.max(), m.min(), m.max()])
plt.colorbar()
plt.show()

circle = patches.Circle((0,0), 12.5, fill=False, color='white', alpha=0.7, linewidth=2)

fig, ax =plt.subplots()
plt.imshow(np.abs(ds.APERTURE[0,chan,0,:,:]),extent=[u.min(), u.max(), v.min(), v.max()])

plt.show()

ds

In [ ]:
from astrohack.panel import panel

panel_model = 'rigid'

before_panel = panel(
    image_name=beforeimage, 
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)
after_panel = panel(
    image_name=afterimage, 
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)

In [ ]:
import numpy as np
from astrohack.dio import open_panel

def relative_difference(mean, expected):  
    return 2*np.abs(mean - expected)/(abs(mean) + abs(expected))

def verify_panel_shifts(
    panel_list=['3-4', '5-27', '5-37', '5-38'], 
    expected_shift=np.array([-100, 75, 0, 150]),
    ref_mean_shift = np.array([-112.23760235, 73.09423151, -1.52957784, 138.96735818]),
    antenna='ant_ea25',
    ddi='ddi_0'
):
    
    M_TO_MILS = 39370.1
    
    before_mds = open_panel('results/before.split.panel.zarr')
    after_mds = open_panel('results/after.split.panel.zarr')
    
    before_shift = before_mds[antenna][ddi].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS
    after_shift = after_mds[antenna][ddi].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS
    
    difference = after_shift - before_shift
    
    mean_shift = np.mean(difference, axis=1)
    
    delta_mean_shift = np.abs(mean_shift - expected_shift)
    delta_ref_shift = np.abs(ref_mean_shift - expected_shift)
        
    delta_shift = delta_mean_shift - delta_ref_shift  # New corrections - old corrections --> delta if delta < 0 ==> we improved.
    relative_shift = relative_difference(delta_mean_shift, delta_ref_shift)
       
    if np.any(relative_shift > 1e-6): 
        print("There were changes!")
        for i, delta in enumerate(delta_shift):
            if delta < 0:
                print("{panel}, improved by {delta} mils".format(panel=panel_list[i], delta=delta))
            else:
                print("{panel}, got worse by {delta} mils".format(panel=panel_list[i], delta=delta))

In [ ]:
verify_panel_shifts()